In [1]:
import json
import time
import random
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def safe_find_element(driver, by, selector, default="정보 없음", attribute=None):
    """단일 요소를 안전하게 추출"""
    try:
        element = driver.find_element(by, selector)
        return element.get_attribute(attribute).strip() if attribute else element.text.strip()
    except Exception:
        return default

def safe_find_elements(driver, by, selector, attribute=None):
    """다중 요소를 안전하게 추출"""
    try:
        elements = driver.find_elements(by, selector)
        if attribute:
            return [el.get_attribute(attribute) for el in elements if el.get_attribute(attribute)]
        return [el.text.strip() for el in elements if el.text.strip()]
    except Exception:
        return []

def scroll_to_load(driver, max_scrolls=10, pause_time=0.2):
    """페이지 끝까지 스크롤하며 콘텐츠 로드"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_sample_place_data(restaurant_name):
    """음식점 정보 크롤링"""
    search_url = f"https://map.naver.com/p/search/{restaurant_name} 강남구"
    
    # Selenium 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('--headless=new')  # 헤드리스 모드 (필요 시 활성화)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 3)  # 타임아웃 3초로 설정

        # entryIframe 확인 및 진입
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        except:
            # 검색 결과 iframe 진입
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
            driver.switch_to.frame("searchIframe")
            
            short_name = re.split(r'\s*\(', restaurant_name)[0]
            # restaurant_name이 들어간 span을 가진 a 태그 클릭 (원래 로직 복원)
            target_xpath = f"//a[@role='button'][.//span[contains(text(), '{short_name}')]]"
            try:
                first_result = driver.find_element(By.XPATH, target_xpath)
                first_result.click()
            except Exception as e:
                print(f"검색 결과에서 '{restaurant_name}'을 찾거나 클릭할 수 없습니다: {e}")
                return None
            
            driver.switch_to.default_content()
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        
        driver.switch_to.frame("entryIframe")

        # 데이터 추출
        data = {
            '음식점_이름': safe_find_element(driver, By.CSS_SELECTOR, "span.GHAhO"),
            '음식점_사진': safe_find_element(driver, By.CSS_SELECTOR, "div.fNygA img", attribute="src"),
            '주소': safe_find_element(driver, By.CSS_SELECTOR, "span.LDgIH"),
            '카테고리': safe_find_element(driver, By.CSS_SELECTOR, "span.lnJFt"),
            '전화번호': safe_find_element(driver, By.CSS_SELECTOR, "span.xlx7Q")
        }

        # 영업시간 추출
        try:
            if not driver.find_elements(By.CSS_SELECTOR, "div.H3ua4"):
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.gKP9i.RMgN0"))).click()
            openhour_rows = driver.find_elements(By.CSS_SELECTOR, "span.A_cdD")
            openhours = []
            for row in openhour_rows:
                try:
                    day = safe_find_element(row, By.CSS_SELECTOR, "span.i8cJw")
                    hours = safe_find_element(row, By.CSS_SELECTOR, "div.H3ua4").replace('\n', '; ')
                    openhours.append(f"{day}: {hours}")
                except Exception:
                    continue
            data['영업시간'] = "; ".join(openhours) if openhours else "정보 없음"
        except Exception:
            data['영업시간'] = "정보 없음"

        # 리뷰 데이터 추출
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "//span[normalize-space(text())='리뷰']"))).click()
            time.sleep(1.0)  # 리뷰 탭 로딩 대기 시간 증가
            
            # 리뷰 데이터 수집 (최대 100개, 10개씩 처리)
            review_data = []
            max_reviews = 50
            batch_size = 10  # 10개씩 처리
            
            # 리뷰 로딩 대기 (최대 5초)
            retry_count = 0
            max_retries = 10
            while retry_count < max_retries:
                review_lis = driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui.EjjAW")
                if review_lis:
                    break  # 리뷰가 로드되면 루프 종료
                time.sleep(0.5)
                retry_count += 1
                print(f"리뷰 로딩 대기 중... ({retry_count}/{max_retries})")
            
            # 리뷰가 없는 경우 확인
            no_review_msg = driver.find_elements(By.XPATH, "//div[contains(text(), '리뷰가 없습니다')]")
            if no_review_msg:
                print("리뷰가 없습니다.")
                data['리뷰'] = json.dumps([], ensure_ascii=False)
                return  # 또는 다음 단계로 진행
            
            # 리뷰가 없으면 빈 배열 반환
            if not review_lis:
                print("리뷰를 찾을 수 없습니다.")
                data['리뷰'] = json.dumps([], ensure_ascii=False)
                return  # 또는 다음 단계로 진행
            
            while len(review_data) < max_reviews:
                try:
                    # 현재 로드된 리뷰 가져오기 (이미 처리된 리뷰 제외)
                    all_reviews = driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui.EjjAW")
                    remaining_reviews = all_reviews[len(review_data):min(len(review_data) + batch_size, max_reviews)]
                    
                    if not remaining_reviews:  # 더 이상 처리할 리뷰가 없으면 종료
                        break
                        
                    # 10개 리뷰 렌더링 및 데이터 추출
                    batch_data = []
                    for li in remaining_reviews:
                        try:
                            # 기존 코드와 동일...
                            # 리뷰 요소로 스크롤
                            driver.execute_script("arguments[0].scrollIntoView(true);", li)
                            time.sleep(0.5)  # 렌더링 대기 증가

                            # 게시글 더보기 버튼 클릭
                            post_more_btn = li.find_elements(By.CSS_SELECTOR, "a[data-pui-click-code='rvshowmore']")
                            if post_more_btn:
                                driver.execute_script("arguments[0].scrollIntoView(true);", post_more_btn[0])
                                time.sleep(0.1)
                                driver.execute_script("arguments[0].click();", post_more_btn[0])
                                time.sleep(0.7)  # 태그 및 게시글 로딩 대기 증가

                            # 데이터 추출
                            tags = safe_find_elements(li, By.CSS_SELECTOR, "div.pui__HLNvmI span.pui__jhpEyP")
                            post = safe_find_element(li, By.CSS_SELECTOR, "div.pui__vn15t2 span, div.pui__vn15t2")
                            if post == "정보 없음":
                                print(f"게시글 추출 실패, 대안 시도: {li.get_attribute('outerHTML')[:100]}")
                                post = safe_find_element(li, By.CSS_SELECTOR, "a[data-pui-click-code='rvshowmore']")  # 대안
                            images = safe_find_elements(li, By.CSS_SELECTOR, "div.HH5sZ img", attribute="src")[:3]
                            keywords = safe_find_elements(li, By.CSS_SELECTOR, "div.pui__-0Ter1 span em")

                            batch_data.append({
                                "태그": tags,
                                "게시글": post,
                                "이미지": images,
                                "키워드": keywords
                            })
                        except Exception as e:
                            print(f"리뷰 데이터 처리 중 오류: {e}")
                            continue

                    # 배치 데이터 저장
                    review_data.extend(batch_data)
                    # print(f"리뷰 {len(review_data)}개 수집 완료")

                    # 다음 10개 로드 (리뷰 더보기 버튼 또는 스크롤)
                    more_btn = driver.find_elements(By.CSS_SELECTOR, "a.fvwqf")  
                    if more_btn:
                        driver.execute_script("arguments[0].scrollIntoView(true);", more_btn[0])
                        time.sleep(0.2)
                        driver.execute_script("arguments[0].click();", more_btn[0])
                        time.sleep(1.0)  # 리뷰 로딩 대기 시간 증가
                    else:
                        driver.execute_script("window.scrollBy(0, 500);")  # 스크롤 거리 증가
                        time.sleep(0.5)  # 대기 시간 증가

                    # 새로운 리뷰가 로드되었는지 확인
                    new_review_count = len(driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui.EjjAW"))
                    if new_review_count <= len(review_data):
                        print("더 이상 새로운 리뷰가 없습니다.")
                        break

                except Exception as e:
                    print(f"리뷰 로드 중 오류: {e}")
                    break

            data['리뷰'] = json.dumps(review_data, ensure_ascii=False)
        except Exception as e:
            print(f"리뷰 추출 중 오류: {e}")
            data['리뷰'] = json.dumps([], ensure_ascii=False)



        # 메뉴 정보 추출
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '메뉴')]"))).click()
            time.sleep(0.2)
            scroll_to_load(driver, max_scrolls=5, pause_time=0.1)

            menus = []
            for selector in [("li.order_list_item", "div.tit", "div.price > strong"), 
                            ("li.E2jtL", "span.lPzHi", "div.GXS1X em")]:
                items = driver.find_elements(By.CSS_SELECTOR, selector[0])
                for item in items:
                    name = safe_find_element(item, By.CSS_SELECTOR, selector[1])
                    price = safe_find_element(item, By.CSS_SELECTOR, selector[2])
                    if name != "정보 없음" and price != "정보 없음":
                        menus.append({"메뉴명": name, "가격": price})

            unique = set()
            deduped_menus = [menu for menu in menus if not (menu['메뉴명'], menu['가격']) in unique and not unique.add((menu['메뉴명'], menu['가격']))]
            data['메뉴_정보'] = json.dumps(deduped_menus, ensure_ascii=False)
        except Exception:
            data['메뉴_정보'] = json.dumps([], ensure_ascii=False)

        return data

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
        return None
    finally:
        driver.quit()

In [2]:
def collect_all_restaurant_data(restaurants_df, start_idx, end_idx):
    for i in range(start_idx, end_idx+1):
        name = restaurants_df.loc[i, '음식점명']
        print('*' * 50)
        print(f'{i}번째 음식점: {name} 수집 시작')

        try:
            data = get_sample_place_data(name)

            if data:
                df_one = pd.DataFrame([data])
                # 파일 경로 설정
                file_path = '../../data/external/gangnam_crawling_restaurant_data.csv'
                # 파일 존재 여부 확인
                header = not os.path.exists(file_path)
                # DataFrame을 CSV로 저장
                df_one.to_csv(file_path, mode='a', header=header, index=False, encoding='utf-8-sig')
                print(f'{i}번째 음식점: {name} 저장 완료')
            else:
                print(f'{i}번째 음식점: {name} 데이터 없음')

        except Exception as e:
            print(f'오류 발생: {e} → {i}번째 음식점: {name} 수집 실패')

        time.sleep(random.uniform(3, 5))


In [ ]:
# CSV 파일에서 음식점 목록 로드
restaurants_df = pd.read_csv('../../data/interim/gangnam_restaurants_cleaned.csv')

# 전체 데이터 수집 (시간이 오래 걸릴 수 있음)
# 시작 인덱스 (gangnam_restaurants_cleaned.csv에서 -2)
start_idx = 4168
end_idx = 6500
result = collect_all_restaurant_data(restaurants_df, start_idx, end_idx)


**************************************************
3982번째 음식점: 오또상 수집 시작
3982번째 음식점: 오또상 저장 완료
**************************************************
3983번째 음식점: 술속의밤 신논현점 수집 시작
리뷰 로딩 대기 중... (1/10)
3983번째 음식점: 술속의밤 신논현점 저장 완료
**************************************************
3984번째 음식점: 츄잇 수집 시작
검색 결과에서 '츄잇'을 찾거나 클릭할 수 없습니다: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@role='button'][.//span[contains(text(), '츄잇')]]"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0041FC83+61635]
	GetHandleVerifier [0x0041FCC4+61700]
	(No symbol) [0x002405D3]
	(No symbol) [0x0028899E]
	(No symbol) [0x00288D3B]
	(No symbol) [0x002D0E12]
	(No symbol) [0x002AD2E4]
	(No symbol) [0x002CE61B]
	(No symbol) [0x002AD096]
	(No symbol) [0x0027C840]
	(No symbol) [0x0027D6A4]
	GetHandleVerifier [0x006A45A3

KeyboardInterrupt: 